<a href="https://colab.research.google.com/github/danielpatrickhug/libcst_examples/blob/main/libcst_node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Nodes
CSTNode and its subclasses cover Python’s full grammar in a whitespace-sensitive fashion, forming LibCST’s concrete syntax tree.

Many of these nodes are designed to behave similarly to [Python’s abstract syntax tree.](https://greentreesnakes.readthedocs.io/en/latest/nodes.html)

[Based](https://libcst.readthedocs.io/en/latest/nodes.html)

### CSTNode

#### with_changes()

This function is used to perform mutation-like operations on immutable nodes. Here is an example of how to use it:

In [ ]:
!pip install libcst

In [10]:
import libcst as cst


def update_conditional(node, new_conditional):
    new_node = node.with_changes(test=new_conditional)
    return new_node

tree = cst.parse_module("if True: pass")
new_conditional = cst.parse_expression("False")
updated_tree = update_conditional(tree.body[0], new_conditional)
print(updated_tree)


If(
    test=Name(
        value='False',
        lpar=[],
        rpar=[],
    ),
    body=SimpleStatementSuite(
        body=[
            Pass(
                semicolon=MaybeSentinel.DEFAULT,
            ),
        ],
        leading_whitespace=SimpleWhitespace(
            value=' ',
        ),
        trailing_whitespace=TrailingWhitespace(
            whitespace=SimpleWhitespace(
                value='',
            ),
            comment=None,
            newline=Newline(
                value=None,
            ),
        ),
    ),
    orelse=None,
    leading_lines=[],
    whitespace_before_test=SimpleWhitespace(
        value=' ',
    ),
    whitespace_after_test=SimpleWhitespace(
        value='',
    ),
)


In [11]:
import libcst as cst

def update_conditional(tree, new_conditional):
    if_node = tree.body[0]
    new_node = if_node.with_changes(test=new_conditional)
    updated_tree = tree.with_changes(body=[new_node])
    return updated_tree

tree = cst.parse_module("if True: pass")
new_conditional = cst.parse_expression("False")
updated_tree = update_conditional(tree, new_conditional)
print(updated_tree.code)


if False: pass


#### deep_clone()

This function is used to make a clone of a CSTNode. Here is an example of how to use it:

In [4]:
import libcst as cst

tree = cst.parse_expression("1+2")
cloned_tree = tree.deep_clone()

print(tree == cloned_tree)  # False: They are not the same object
print(tree.deep_equals(cloned_tree))  # True: They are equivalent trees


False
True


#### deep_replace()

This function is used to replace a node in the CST. Here is an example of how to use it:

In [5]:
import libcst as cst

tree = cst.parse_module("x = 1\ny = 2")
old_node = tree.body[0]  # the node representing 'x = 1'
new_node = cst.parse_statement("x = 3")  # new statement node

new_tree = tree.deep_replace(old_node, new_node)
print(new_tree.code)


x = 3
y = 2


#### deep_remove()

This function is used to remove a node from the CST. Here is an example of how to use it:

In [6]:
import libcst as cst

tree = cst.parse_module("x = 1\ny = 2")
node_to_remove = tree.body[0]  # the node representing 'x = 1'

new_tree = tree.deep_remove(node_to_remove)
print(new_tree.code)


y = 2


## Modules
A node that represents an entire python module.

### visit()

This function is used to run a visitor over the module. Here's an example of how you can use it:

In [12]:
import libcst as cst
import libcst.matchers as m

class FunctionNameVisitor(cst.CSTVisitor):
    def visit_FunctionDef(self, node: cst.FunctionDef) -> None:
        print("Function name:", node.name.value)

module = cst.parse_module("def foo(): pass\ndef bar(): pass")
module.visit(FunctionNameVisitor())


Function name: foo
Function name: bar


Module(
    body=[
        FunctionDef(
            name=Name(
                value='foo',
                lpar=[],
                rpar=[],
            ),
            params=Parameters(
                params=[],
                star_arg=MaybeSentinel.DEFAULT,
                kwonly_params=[],
                star_kwarg=None,
                posonly_params=[],
                posonly_ind=MaybeSentinel.DEFAULT,
            ),
            body=SimpleStatementSuite(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_whitespace=SimpleWhitespace(
                    value=' ',
                ),
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
          

### code property

This property gives the string representation of the module. Here's how you can use it:

In [13]:
import libcst as cst

module = cst.parse_module("def foo(): pass\ndef bar(): pass")
print(module.code)


def foo(): pass
def bar(): pass


### bytes property

This property gives the bytes representation of the module. Here's how you can use it:

In [14]:
import libcst as cst

module = cst.parse_module("def foo(): pass\ndef bar(): pass")
print(module.bytes)


b'def foo(): pass\ndef bar(): pass'


### code_for_node()

This method generates the code for a given node in the context of the module:

In [15]:
import libcst as cst

module = cst.parse_module("def foo(): pass\ndef bar(): pass")
function_node = module.body[0]
print(module.code_for_node(function_node))


def foo(): pass



### config_for_parsing property

This property generates a parser config suitable for passing to a *parse_expression()* or *parse_statement()* call:

In [16]:
import libcst as cst

module = cst.parse_module("def foo(): pass\ndef bar(): pass")
expression = cst.parse_expression("1 + 2", config=module.config_for_parsing)
print(expression)


BinaryOperation(
    left=Integer(
        value='1',
        lpar=[],
        rpar=[],
    ),
    operator=Add(
        whitespace_before=SimpleWhitespace(
            value=' ',
        ),
        whitespace_after=SimpleWhitespace(
            value=' ',
        ),
    ),
    right=Integer(
        value='2',
        lpar=[],
        rpar=[],
    ),
    lpar=[],
    rpar=[],
)


### get_docstring()

This method returns a cleaned docstring if the docstring is available:

In [17]:
source_code = """
\"\"\"
This is a docstring.
\"\"\"

def foo():
    pass

def bar(x, y):
    return x + y
"""
module = cst.parse_module(source_code)
print(module.get_docstring())

This is a docstring.


## Expressions
An expression is anything that represents a value (e.g. it could be returned from a function). All expressions subclass from BaseExpression.

Expression can be parsed with parse_expression() or as part of a statement or module using parse_statement() or parse_module().

### Names and Object Attributes:

In [42]:
import libcst as cst

# Name
name = cst.Name(value="x")
#print(name)
print(cst.Module([name]).code)


# Attribute
attribute = cst.Attribute(value=cst.Name("x"), attr=cst.Name("y"))
#print(attribute)
print(cst.Module([attribute]).code)



x
x.y


### Operations and Comparisons:

In [41]:
# UnaryOperation
unary_operation = cst.UnaryOperation(operator=cst.Not(), expression=cst.Name("x"))
#print(unary_operation) #print CST
print(cst.Module([unary_operation]).code)


# BinaryOperation
binary_operation = cst.BinaryOperation(left=cst.Name("x"), operator=cst.Add(), right=cst.Name("y"))
#print(binary_operation) #print CST
print(cst.Module([binary_operation]).code)


# BooleanOperation
boolean_operation = cst.BooleanOperation(left=cst.Name("x"), operator=cst.And(), right=cst.Name("y"))
#print(boolean_operation) #print CST
print(cst.Module([boolean_operation]).code)


# Comparison
comparison = cst.Comparison(
    left=cst.Name("x"), 
    comparisons=[
        cst.ComparisonTarget(operator=cst.LessThan(), comparator=cst.Name("y")),
        cst.ComparisonTarget(operator=cst.LessThan(), comparator=cst.Name("z"))
    ]
)
#print(comparison) #print CST
print(cst.Module([comparison]).code)


not x
x + y
x and y
x < y < z


### Control Flow

In [37]:
# Asynchronous
asynchronous = cst.Asynchronous()
#print(asynchronous) #print CST
print(cst.Module([asynchronous]).code)

# Await
await_expr = cst.Await(expression=cst.Name("x"))
#print(await_expr) #print CST
print(cst.Module([await_expr]).code)

# Yield
yield_expr = cst.Yield(value=cst.Name("x"))
#print(yield_expr) #print CST
print(cst.Module([yield_expr]).code)


# From
from_expr = cst.From(item=cst.Name("x"))
#print(from_expr) #print CST
print(cst.Module([from_expr]).code)

# IfExp
if_exp = cst.IfExp(test=cst.Name("x"), body=cst.Name("y"), orelse=cst.Name("z"))
#print(if_exp) #print CST
print(cst.Module([if_exp]).code)




async 
await x
yield x
from x
y if x else z


### Lambdas and Function Calls:

In [33]:
# Lambda
lambda_expr = cst.Lambda(
    params=cst.Parameters(params=[cst.Param(name=cst.Name("arg"))]), 
    body=cst.Ellipsis()
)
#print(lambda_expr)
print(cst.Module([lambda_expr]).code)


# Call
call_expr = cst.Call(
    func=cst.Name("do_math"), 
    args=[
        cst.Arg(value=cst.Integer("1")),
        cst.Arg(value=cst.Integer("2"))
    ]
)
#print(call_expr)
print(cst.Module([call_expr]).code)

# Arg
arg = cst.Arg(value=cst.Name("x"))
print(cst.Module([arg]).code)


lambda arg: ...
do_math(1, 2)
x


In [43]:
import libcst as cst

# Define an async web server route
async_def = cst.FunctionDef(
    name=cst.Name("handle"),
    params=cst.Parameters(params=[
        cst.Param(name=cst.Name("request"))
    ]),
    body=cst.IndentedBlock(
        body=[
            cst.SimpleStatementLine(
                body=[
                    cst.Expr(
                        cst.Await(
                            expression=cst.Call(
                                func=cst.Attribute(
                                    value=cst.Name("request"),
                                    attr=cst.Name("text")
                                )
                            )
                        )
                    )
                ]
            ),
            cst.If(
                test=cst.Comparison(
                    left=cst.Name("request"),
                    comparisons=[
                        cst.ComparisonTarget(
                            operator=cst.Equal(),
                            comparator=cst.SimpleString('"shutdown"')
                        )
                    ]
                ),
                body=cst.IndentedBlock(
                    body=[
                        cst.SimpleStatementLine(
                            body=[
                                cst.Expr(
                                    cst.Await(
                                        expression=cst.Call(
                                            func=cst.Name("shutdown"),
                                            args=[
                                                cst.Arg(value=cst.Name("request"))
                                            ]
                                        )
                                    )
                                )
                            ]
                        )
                    ]
                ),
                orelse=cst.IndentedBlock(
                    body=[
                        cst.SimpleStatementLine(
                            body=[
                                cst.Return(
                                    value=cst.Call(
                                        func=cst.Attribute(
                                            value=cst.Attribute(
                                                value=cst.Name("aiohttp"),
                                                attr=cst.Name("web")
                                            ),
                                            attr=cst.Name("Response")
                                        ),
                                        args=[
                                            cst.Arg(
                                                keyword=cst.Name("text"), 
                                                equal=cst.AssignEqual(), 
                                                value=cst.SimpleString('"Hello, world!"')
                                            )
                                        ]
                                    )
                                )
                            ]
                        )
                    ]
                )
            )
        ]
    ),
    returns=None,
    asynchronous=cst.Asynchronous()
)

# Create a module with the function
module = cst.Module([async_def])

# Print the generated code
print(module.code)


async def handle(request):
    await request.text()
    if request == "shutdown":
        await shutdown(request)

        return aiohttp.web.Response(text = "Hello, world!")



### Literal Values

#### Integer Literals

In [44]:
from libcst import Integer, Module

int_expr = Integer("123")
# print(int_expr)
print(Module([int_expr]).code)


123


#### Float Literals


In [45]:
from libcst import Float, Module

float_expr = Float("123.456")
# print(float_expr)
print(Module([float_expr]).code)


123.456


#### Imaginary Literals

In [46]:
from libcst import Imaginary, Module

imag_expr = Imaginary("123j")
# print(imag_expr)
print(Module([imag_expr]).code)


123j


#### SimpleString Literal

In [47]:
from libcst import SimpleString, Module

string_expr = SimpleString("'Hello, world!'")
# print(string_expr)
print(Module([string_expr]).code)


'Hello, world!'


#### ConcatenatedString Literal

In [48]:
from libcst import ConcatenatedString, SimpleString, Module

concat_expr = ConcatenatedString(
    left=SimpleString("'Hello,'"),
    right=SimpleString("' world!'"),
    lpar=[],
    rpar=[],
    whitespace_between=cst.SimpleWhitespace(' ')
)
# print(concat_expr)
print(Module([concat_expr]).code)


'Hello,' ' world!'


#### FormattedString Literal

In [49]:
from libcst import FormattedString, FormattedStringText, FormattedStringExpression, Name, Module

fstring_expr = FormattedString(
    parts=[
        FormattedStringText(value="Hello, "),
        FormattedStringExpression(
            expression=Name("name"),
            conversion=None,
            format_spec=None,
            whitespace_before_expression=cst.SimpleWhitespace(''),
            whitespace_after_expression=cst.SimpleWhitespace(''),
        ),
        FormattedStringText(value="!"),
    ],
    start='f"',
    end='"',
    lpar=[],
    rpar=[],
)
# print(fstring_expr)
print(Module([fstring_expr]).code)


f"Hello, {name}!"


### Collections

#### Tuples

In [51]:
from libcst import Tuple, Element, Integer, Name, StarredElement

tuple_expr = cst.Module([
    cst.Expr(
        Tuple([
            Element(Integer("1")),
            Element(Integer("2")),
            StarredElement(Name("others")),
        ])
    )
])

# print(tuple_expr)
print(tuple_expr.code)


(1, 2, *others)


#### Lists

In [52]:
from libcst import List, Element, Integer, Name, StarredElement

list_expr = cst.Module([
    cst.Expr(
        List([
            Element(Integer("1")),
            Element(Integer("2")),
            StarredElement(Name("others")),
        ])
    )
])

# print(list_expr)
print(list_expr.code)


[1, 2, *others]


#### Sets

In [53]:
from libcst import Set, Element, Integer, Name, StarredElement

set_expr = cst.Module([
    cst.Expr(
        Set([
            Element(Integer("1")),
            Element(Integer("2")),
            StarredElement(Name("others")),
        ])
    )
])

# print(set_expr)
print(set_expr.code)


{1, 2, *others}


#### Dictionaries

In [54]:
from libcst import Dict, DictElement, Name, StarredDictElement

dict_expr = cst.Module([
    cst.Expr(
        Dict([
            DictElement(Name("k1"), Name("v1")),
            DictElement(Name("k2"), Name("v2")),
            StarredDictElement(Name("expanded")),
        ])
    )
])

# print(dict_expr)
print(dict_expr.code)


{k1: v1, k2: v2, **expanded}


### Comprehensions

#### List Comprehension

In [59]:
import libcst as cst

list_comp_node = cst.ListComp(
    elt=cst.Name("x"),
    for_in=cst.CompFor(
        target=cst.Name("x"),
        iter=cst.Call(
            func=cst.Name("range"),
            args=[
                cst.Arg(cst.Integer("10"))
            ]
        )
    )
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=list_comp_node)])])
print(code.code)
print(eval(code.code))


[x for x in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


#### Set Comprehension

In [61]:
import libcst as cst

set_comp_node = cst.SetComp(
    elt=cst.Name("x"),
    for_in=cst.CompFor(
        target=cst.Name("x"),
        iter=cst.Call(
            func=cst.Name("range"),
            args=[
                cst.Arg(cst.Integer("10"))
            ]
        )
    )
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=set_comp_node)])])
print(code.code)
print(eval(code.code))

{x for x in range(10)}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


#### DictComprehension

In [62]:
import libcst as cst

dict_comp_node = cst.DictComp(
    key=cst.Name("x"),
    value=cst.BinaryOperation(
        left=cst.Name("x"),
        operator=cst.Multiply(),
        right=cst.Integer("2")
    ),
    for_in=cst.CompFor(
        target=cst.Name("x"),
        iter=cst.Call(
            func=cst.Name("range"),
            args=[
                cst.Arg(cst.Integer("10"))
            ]
        )
    )
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=dict_comp_node)])])
print(code.code)
print(eval(code.code))

{x: x * 2 for x in range(10)}

{0: 0, 1: 2, 2: 4, 3: 6, 4: 8, 5: 10, 6: 12, 7: 14, 8: 16, 9: 18}


#### Generator Expression

In [63]:
import libcst as cst

gen_exp_node = cst.GeneratorExp(
    elt=cst.Name("x"),
    for_in=cst.CompFor(
        target=cst.Name("x"),
        iter=cst.Call(
            func=cst.Name("range"),
            args=[
                cst.Arg(cst.Integer("10"))
            ]
        )
    )
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=gen_exp_node)])])
print(code.code)
print(eval(code.code))


(x for x in range(10))

<generator object <genexpr> at 0x7f2286803760>


### Subscripts and Slices

#### Subscript

In [66]:
import libcst as cst

subscript_node = cst.Subscript(
    value=cst.Name("my_list"),
    slice=[
        cst.SubscriptElement(
            slice=cst.Index(value=cst.Integer("2"))
        )
    ]
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=subscript_node)])])
print(code.code)


my_list[2]



#### Slice

In [67]:
import libcst as cst

slice_node = cst.Subscript(
    value=cst.Name("my_list"),
    slice=[
        cst.SubscriptElement(
            slice=cst.Slice(
                lower=cst.Integer("1"),
                upper=cst.Integer("5"),
                step=None,
                first_colon=cst.Colon(),
                second_colon=cst.MaybeSentinel.DEFAULT
            )
        )
    ]
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=slice_node)])])
print(code.code)


my_list[1:5]



#### Extended Slice

In [68]:
import libcst as cst

ext_slice_node = cst.Subscript(
    value=cst.Name("my_arr"),
    slice=[
        cst.SubscriptElement(
            slice=cst.Index(value=cst.Integer("0"))
        ),
        cst.SubscriptElement(
            slice=cst.Slice(
                lower=cst.Integer("1"),
                upper=cst.Integer("5"),
                step=None,
                first_colon=cst.Colon(),
                second_colon=cst.MaybeSentinel.DEFAULT
            ),
            comma=cst.Comma()
        )
    ]
)

code = cst.Module(body=[cst.SimpleStatementLine(body=[cst.Expr(value=ext_slice_node)])])
print(code.code)


my_arr[0, 1:5,]



### Parenthesis, Brackets, and Braces

#### LeftParen and RightParen

In [75]:
import libcst as cst

# Create a function call
call = cst.Call(
    func=cst.Name("my_func"),
    args=[
        cst.Arg(cst.Integer("5")),
    ],
    lpar=[cst.LeftParen(whitespace_after=cst.SimpleWhitespace(value=" "))],
    rpar=[cst.RightParen(whitespace_before=cst.SimpleWhitespace(value=" "))]
)

# Create a module and print the generated code
module = cst.Module(body=[cst.Expr(call)])
print(module.code)


( my_func(5) )


#### LeftSquareBracket and RightSquareBracket 

In [76]:
import libcst as cst

# Create a list
my_list = cst.List(
    elements=[
        cst.Element(value=cst.Integer("1")),
        cst.Element(value=cst.Integer("2")),
    ],
    lbracket=cst.LeftSquareBracket(whitespace_after=cst.SimpleWhitespace(value=" ")),
    rbracket=cst.RightSquareBracket(whitespace_before=cst.SimpleWhitespace(value=" "))
)

# Create a module and print the generated code
module = cst.Module(body=[cst.Expr(my_list)])
print(module.code)


[ 1, 2 ]


#### LeftCurlyBrace and RightCurlyBrace

In [77]:
import libcst as cst

# Create a dictionary
my_dict = cst.Dict(
    elements=[
        cst.DictElement(
            key=cst.SimpleString('"key"'),
            value=cst.Integer("1")
        ),
    ],
    lbrace=cst.LeftCurlyBrace(whitespace_after=cst.SimpleWhitespace(value=" ")),
    rbrace=cst.RightCurlyBrace(whitespace_before=cst.SimpleWhitespace(value=" "))
)

# Create a module and print the generated code
module = cst.Module(body=[cst.Expr(my_dict)])
print(module.code)


{ "key": 1 }


## Statements
Statements represent a “line of code” or a control structure with other lines of code, such as an If block.

All statements subclass from BaseSmallStatement or BaseCompoundStatement.

Statements can be parsed with parse_statement() or as part of a module using parse_module().

### Simple Statements

#### AnnAssign


In [84]:
import libcst as cst

ann_assign = cst.AnnAssign(
    target=cst.Name("x"),
    annotation=cst.Annotation(cst.Name("int")),
    value=cst.Integer("5"),
)
print(cst.Module([cst.SimpleStatementLine([ann_assign])]).code)



x: int = 5



#### Assert


In [85]:
assert_statement = cst.Assert(
    test=cst.BinaryOperation(
        left=cst.Name("x"),
        operator=cst.GreaterThan(),
        right=cst.Integer("5"),
    )
)
print(cst.Module([assert_statement]).code)


assert x > 5


#### Assign


In [86]:

assign_statement = cst.Assign(
    targets=[cst.AssignTarget(target=cst.Name("x"))],
    value=cst.Name("y"),
)
print(cst.Module([assign_statement]).code)


x = y


#### AugAssign


In [87]:

aug_assign = cst.AugAssign(
    target=cst.Name("x"),
    operator=cst.AddAssign(),
    value=cst.Integer("5"),
)
print(cst.Module([aug_assign]).code)


x += 5


#### Break


In [88]:

break_statement = cst.Break()
print(cst.Module([break_statement]).code)


break


#### Continue


In [89]:

continue_statement = cst.Continue()
print(cst.Module([continue_statement]).code)


continue


#### Del


In [90]:

del_statement = cst.Del(cst.Name("x"))
print(cst.Module([del_statement]).code)


del x


#### Expr (function call)


In [105]:

expr_statement = cst.Expr(cst.Call(func=cst.Name("print"), args=[cst.Arg(cst.Name("i"))]))
print(cst.Module([expr_statement]).code)


print(i)


#### Global


In [92]:

global_statement = cst.Global(names=[cst.NameItem(name=cst.Name("x"))])
print(cst.Module([global_statement]).code)


global x


#### Import


In [93]:

import_statement = cst.Import(names=[cst.ImportAlias(name=cst.Name("os"))])
print(cst.Module([import_statement]).code)


import os


#### ImportFrom


In [94]:

import_from_statement = cst.ImportFrom(
    module=cst.Name("os"),
    names=[cst.ImportAlias(name=cst.Name("path"))],
)
print(cst.Module([import_from_statement]).code)


from os import path


#### Nonlocal


In [95]:

nonlocal_statement = cst.Nonlocal(names=[cst.NameItem(name=cst.Name("x"))])
print(cst.Module([nonlocal_statement]).code)


nonlocal x


#### Pass


In [96]:

pass_statement = cst.Pass()
print(cst.Module([pass_statement]).code)


pass


#### Raise


In [97]:

raise_statement = cst.Raise(exc=cst.Call(func=cst.Name("Exception"), args=[]))
print(cst.Module([raise_statement]).code)


raise Exception()


#### Return


In [98]:

return_statement = cst.Return(value=cst.Name("x"))
print(cst.Module([return_statement]).code)

return x


### Compound Statements
Statements that have one or more statement blocks as a child attribute.

#### ClassDef

In [99]:
class_def = cst.ClassDef(
    name=cst.Name("TestClass"),
    bases=[],
    keywords=[],
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([class_def]).code)


class TestClass:
    pass



#### For

In [111]:
for_loop = cst.For(
    target=cst.Name("i"),
    iter=cst.Call(
        func=cst.Name("range"),
        args=[cst.Arg(cst.Integer("10"))]
    ),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[expr_statement])]
    ),
)
print(cst.Module([for_loop]).code)
print(exec(cst.Module([for_loop]).code))


for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9
None


#### FunctionDef

In [112]:
func_def = cst.FunctionDef(
    name=cst.Name("func"),
    params=cst.Parameters(params=[]),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([func_def]).code)


def func():
    pass



#### w/ decorators

In [114]:
func_def_with_decorator = cst.FunctionDef(
    name=cst.Name("func"),
    params=cst.Parameters(params=[]),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    decorators=[
        cst.Decorator(
            decorator=cst.Name("staticmethod")
        )
    ],
)

print(cst.Module([func_def_with_decorator]).code)


@staticmethod
def func():
    pass



#### complex example

In [125]:
import libcst as cst
import libcst.matchers as m

function_def = cst.FunctionDef(
    name=cst.Name("complex_function"),
    params=cst.Parameters(
        params=[
            cst.Param(
                name=cst.Name("arg1"),
                annotation=cst.Annotation(annotation=cst.Name("int")),
                equal=cst.AssignEqual(),
                default=cst.Integer("1"),
            ),
            cst.Param(
                name=cst.Name("arg2"),
                annotation=cst.Annotation(annotation=cst.Name("str")),
                equal=cst.AssignEqual(),
                default=cst.SimpleString("'default'"),
            ),
        ]
    ),
    body=cst.IndentedBlock(
        body=[
            cst.SimpleStatementLine(body=[cst.Pass()])
        ]
    ),
    decorators=[
        cst.Decorator(
            decorator=cst.Name("some_decorator"),

        )
    ],
    returns=cst.Annotation(annotation=cst.Name("None")),
    asynchronous=cst.Asynchronous(),
)

module = cst.Module([function_def])

print(module.code)


@some_decorator
async def complex_function(arg1: int = 1, arg2: str = 'default') -> None:
    pass



#### if

In [113]:
if_stmt = cst.If(
    test=cst.Name("True"),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([if_stmt]).code)


if True:
    pass



#### try

In [126]:
try_stmt = cst.Try(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    handlers=[
        cst.ExceptHandler(
            body=cst.IndentedBlock(
                body=[cst.SimpleStatementLine(body=[cst.Pass()])]
            ),
        )
    ],
)

print(cst.Module([try_stmt]).code)


try:
    pass
except :
    pass



#### while

In [127]:
while_loop = cst.While(
    test=cst.Name("True"),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([while_loop]).code)


while True:
    pass



#### with

In [130]:
with_stmt = cst.With(
    items=[
        cst.WithItem(
            item=cst.Call(func=cst.Name("open"), args=[cst.Arg(cst.SimpleString('"file.txt"'))]),
            asname=cst.AsName(name=cst.Name("f")),
        )
    ],
    body=cst.IndentedBlock(
        body=[
            cst.SimpleStatementLine(body=[cst.Pass()])
        ]
    )
)


print(cst.Module([with_stmt]).code)


with open("file.txt") as f:
    pass



### Helper Nodes
Nodes that are used by various statements to represent some syntax, but are not statements on their own and cannot be used outside of the statements they belong with.

#### Annotations

In [133]:
import libcst as cst

annotation = cst.Annotation(
    annotation=cst.Subscript(
        value=cst.Name("List"),
        slice=[
            cst.SubscriptElement(
                slice=cst.Index(value=cst.Name("str"))
            )
        ]
    ),
)

func_def = cst.FunctionDef(
    name=cst.Name("my_function"),
    params=cst.Parameters(
        params=[
            cst.Param(
                name=cst.Name("param1"),
                annotation=annotation,
            ),
        ]
    ),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([func_def]).code)


def my_function(param1: List[str]):
    pass



#### AsName

In [132]:
import libcst as cst

asname = cst.AsName(
    name=cst.Name("my_alias"),
    whitespace_before_as=cst.SimpleWhitespace(value=" "),
    whitespace_after_as=cst.SimpleWhitespace(value=" "),
)

with_stmt = cst.With(
    items=[
        cst.WithItem(
            item=cst.Call(
                func=cst.Name("open"),
                args=[cst.Arg(value=cst.SimpleString("'file.txt'"))],
            ),
            asname=asname
        ),
    ],
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    )
)

print(cst.Module([with_stmt]).code)


with open('file.txt') as my_alias:
    pass



#### AssignTarget

In [134]:
import libcst as cst

assign_target = cst.AssignTarget(
    target=cst.Name("x"),
    whitespace_before_equal=cst.SimpleWhitespace(" "),
    whitespace_after_equal=cst.SimpleWhitespace(" ")
)

assign = cst.Assign(targets=[assign_target], value=cst.Name("y"))
print(cst.Module([assign]).code)


x = y


#### Decorators


In [135]:
decorator = cst.Decorator(
    decorator=cst.Name("staticmethod"),
    whitespace_after_at=cst.SimpleWhitespace(""),
    trailing_whitespace=cst.TrailingWhitespace(
        whitespace=cst.SimpleWhitespace(" "),
        comment=None,
        newline=cst.Newline()
    )
)

func_def = cst.FunctionDef(
    name=cst.Name("my_function"),
    decorators=[decorator],
    params=cst.Parameters(),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
)

print(cst.Module([func_def]).code)


@staticmethod 
def my_function():
    pass



#### Else

In [140]:
else_clause = cst.Else(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    whitespace_before_colon=cst.SimpleWhitespace(" "),
)

if_stmt = cst.If(
    test=cst.Name("condition"),
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    orelse=else_clause
)

print(cst.Module([if_stmt]).code)


if condition:
    pass
else :
    pass



#### ExceptHandler

In [137]:
except_clause = cst.ExceptHandler(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    type=cst.Name("Exception"),
    whitespace_after_except=cst.SimpleWhitespace(" "),
    whitespace_before_colon=cst.SimpleWhitespace(" "),
)

try_stmt = cst.Try(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    handlers=[except_clause]
)

print(cst.Module([try_stmt]).code)


try:
    pass
except Exception :
    pass



#### Finally

In [138]:
finally_clause = cst.Finally(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    whitespace_before_colon=cst.SimpleWhitespace(" "),
)

try_stmt = cst.Try(
    body=cst.IndentedBlock(
        body=[cst.SimpleStatementLine(body=[cst.Pass()])]
    ),
    finalbody=finally_clause
)

print(cst.Module([try_stmt]).code)


try:
    pass
finally :
    pass



#### ImportAlias

In [141]:
import_alias = cst.ImportAlias(
    name=cst.Name("math"),
    asname=cst.AsName(
        name=cst.Name("m"),
        whitespace_before_as=cst.SimpleWhitespace(" "),
        whitespace_after_as=cst.SimpleWhitespace(" ")
    )
)

import_stmt = cst.Import(names=[import_alias])
print(cst.Module([import_stmt]).code)


import math as m


#### NameItem

In [143]:
name_item = cst.NameItem(
    name=cst.Name("var"), 
)

global_stmt = cst.Global(names=[name_item])
print(cst.Module([global_stmt]).code)


global var


### Statement Blocks
Nodes that represent some group of statements.

#### SimpleStatementLine

In [156]:
import libcst as cst

# Define multiple simple statements on a single line
simple_statement_line = cst.SimpleStatementLine(
    body=[
        cst.Assign(
            targets=[cst.AssignTarget(target=cst.Name("x"))],
            value=cst.Integer("1"),
        ),
        cst.Assign(
            targets=[cst.AssignTarget(target=cst.Name("y"))],
            value=cst.Integer("2"),
        ),
    ]
)

# Print the corresponding code
print(cst.Module([simple_statement_line]).code)
# Output: x = 1; y = 2


x = 1; y = 2



#### SimpleStatementSuite

In [157]:
# Define a simple statement suite as part of an if statement
simple_statement_suite = cst.SimpleStatementSuite(
    body=[
        cst.Assign(
            targets=[cst.AssignTarget(target=cst.Name("x"))],
            value=cst.Integer("1"),
        ),
        cst.Assign(
            targets=[cst.AssignTarget(target=cst.Name("y"))],
            value=cst.Integer("2"),
        ),
    ]
)

# Create an if statement with the simple statement suite
if_statement = cst.If(
    test=cst.Name("condition"),
    body=simple_statement_suite,
)

# Print the corresponding code
print(cst.Module([if_statement]).code)
# Output: if condition: x = 1; y = 2


if condition: x = 1; y = 2



#### IndentedBlock

In [159]:
# Define an indented block as part of an if-else statement
indented_block_if = cst.IndentedBlock(
    body=[
        cst.SimpleStatementLine(
            body=[
                cst.Assign(
                    targets=[cst.AssignTarget(target=cst.Name("x"))],
                    value=cst.Integer("1"),
                )
            ]
        )
    ]
)

indented_block_else = cst.IndentedBlock(
    body=[
        cst.SimpleStatementLine(
            body=[
                cst.Assign(
                    targets=[cst.AssignTarget(target=cst.Name("x"))],
                    value=cst.Integer("2"),
                )
            ]
        )
    ]
)

# Create an if-else statement with the indented blocks
if_statement = cst.If(
    test=cst.Name("condition"),
    body=indented_block_if,
    orelse=cst.Else(
        body=indented_block_else,
    ),
)

# Print the corresponding code
print(cst.Module([if_statement]).code)



if condition:
    x = 1
else:
    x = 2



## Operators

### Unary Operators

In [161]:
import libcst as cst

# Example of using Not operator
not_op = cst.UnaryOperation(
    operator=cst.Not(),
    expression=cst.Name("condition")
)

print(cst.Module([not_op]).code)
# Output: not condition


not condition


### Boolean Operators

In [162]:
# Example of using And operator
and_op = cst.BooleanOperation(
    left=cst.Name("condition1"),
    operator=cst.And(),
    right=cst.Name("condition2"),
)

print(cst.Module([and_op]).code)
# Output: condition1 and condition2


condition1 and condition2


### Binary Operators

In [163]:
# Example of using Add operator
add_op = cst.BinaryOperation(
    left=cst.Integer("1"),
    operator=cst.Add(),
    right=cst.Integer("2"),
)

print(cst.Module([add_op]).code)
# Output: 1 + 2


1 + 2


In [166]:
# (a + b) * (c - d) / e
import libcst as cst

arith_expression = cst.BinaryOperation(
    left=cst.BinaryOperation(
        left=cst.Name("a"),
        operator=cst.Add(),
        right=cst.Name("b")
    ),
    operator=cst.Multiply(),
    right=cst.BinaryOperation(
        left=cst.Name("c"),
        operator=cst.Subtract(),
        right=cst.Name("d")
    )
)

arith_expression = cst.BinaryOperation(
    left=arith_expression,
    operator=cst.Divide(),
    right=cst.Name("e")
)

print(cst.Module([arith_expression]).code)
# Output: (a + b) * (c - d) / e


a + b * c - d / e


In [169]:
import libcst as cst

# A @ B
matrix_mul = cst.BinaryOperation(
    left=cst.Name("A"),
    operator=cst.MatrixMultiply(),
    right=cst.Name("B"),
)

print(cst.Module([matrix_mul]).code)
# Output: A @ B


A @ B


In [170]:
# C = A @ B + D
matrix_mul_and_add = cst.Assign(
    targets=[cst.AssignTarget(target=cst.Name("C"))],
    value=cst.BinaryOperation(
        left=cst.BinaryOperation(
            left=cst.Name("A"),
            operator=cst.MatrixMultiply(),
            right=cst.Name("B"),
        ),
        operator=cst.Add(),
        right=cst.Name("D"),
    ),
)

print(cst.Module([matrix_mul_and_add]).code)
# Output: C = A @ B + D


C = A @ B + D


### Comparison Operators

In [164]:
# Example of using GreaterThan operator
gt_op = cst.Comparison(
    left=cst.Integer("1"),
    comparisons=[
        cst.ComparisonTarget(
            operator=cst.GreaterThan(),
            comparator=cst.Integer("0"),
        )
    ]
)

print(cst.Module([gt_op]).code)
# Output: 1 > 0


1 > 0


In [167]:
# (a > b) and (c < d) or (e == f)
comp_expression = cst.BooleanOperation(
    left=cst.BooleanOperation(
        left=cst.Comparison(
            left=cst.Name("a"),
            comparisons=[
                cst.ComparisonTarget(
                    operator=cst.GreaterThan(),
                    comparator=cst.Name("b")
                )
            ]
        ),
        operator=cst.And(),
        right=cst.Comparison(
            left=cst.Name("c"),
            comparisons=[
                cst.ComparisonTarget(
                    operator=cst.LessThan(),
                    comparator=cst.Name("d")
                )
            ]
        )
    ),
    operator=cst.Or(),
    right=cst.Comparison(
        left=cst.Name("e"),
        comparisons=[
            cst.ComparisonTarget(
                operator=cst.Equal(),
                comparator=cst.Name("f")
            )
        ]
    )
)

print(cst.Module([comp_expression]).code)
# Output: (a > b) and (c < d) or (e == f)


a > b and c < d or e == f


### Augmented Assignment Operators

In [165]:
# Example of using AddAssign operator
add_assign_op = cst.AugAssign(
    target=cst.Name("x"),
    operator=cst.AddAssign(),
    value=cst.Integer("1"),
)

print(cst.Module([add_assign_op]).code)
# Output: x += 1


x += 1


In [168]:
# x = -((a + b) / 2)
# x *= y
unary_assign_expression = cst.SimpleStatementLine(
    body=[
        cst.Assign(
            targets=[cst.AssignTarget(target=cst.Name("x"))],
            value=cst.UnaryOperation(
                operator=cst.Minus(),
                expression=cst.BinaryOperation(
                    left=cst.BinaryOperation(
                        left=cst.Name("a"),
                        operator=cst.Add(),
                        right=cst.Name("b")
                    ),
                    operator=cst.Divide(),
                    right=cst.Integer("2")
                )
            )
        ),
        cst.AugAssign(
            target=cst.Name("x"),
            operator=cst.MultiplyAssign(),
            value=cst.Name("y")
        )
    ]
)

print(cst.Module([unary_assign_expression]).code)
# Output: x = -((a + b) / 2); x *= y


x = -a + b / 2; x *= y

